### Hybrid Search Langchain

In [5]:
!pip install --upgrade --quiet  pinecone pinecone-text pinecone-notebooks

In [3]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [6]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
## initialize the Pinecone client
pc=Pinecone(api_key=api_key)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of dense model
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

d:\Data_Science\01_Projects\GenAI Projects\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
index=pc.Index(index_name)
index

In [8]:
## vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [9]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()
bm25_encoder

In [20]:
sentences=[
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
          "In 2021, I visited New Orleans",

]

## tfidf values on these sentence
bm25_encoder.fit(sentences)

## store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")


100%|██████████| 3/3 [00:00<00:00, 2995.22it/s]


In [21]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)

In [22]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000001C19840CA60>, index=<pinecone.data.index.Index object at 0x000001C196CD1330>)

In [25]:
retriever.add_texts(
    texts=[
        "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",
    ],
    metadatas=[{"year": 2023}, {"year": 2022}, {"year": 2021}],
    ids=["1", "2", "3"]  # Unique IDs required for indexing
)


100%|██████████| 1/1 [00:01<00:00,  1.69s/it]


In [32]:
results = retriever.invoke("what city did I visit first?")

# Extract the top-ranked document
if results:
    top_result = results[0].page_content  # Get the most relevant result
    print("Answer:", top_result)
else:
    print("No relevant results found.")

Answer: In 2021, I visited New Orleans
